In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import copy
import csv

In [2]:
from utils import Data_Read
from Config import Config
from Model import Intere_Gene_Model
%load_ext autoreload
%autoreload 2
config = Config()

In [3]:
train_list_np, test_x, test_y = Data_Read(config)
train_loader_list = []
for train_data in train_list_np:
    train_set = TensorDataset(torch.tensor(train_data))
    train_loader = DataLoader(dataset=train_set, batch_size=config.batch_size,
                          shuffle=True, num_workers=0)
    train_loader_list.append(train_loader)
test_set = TensorDataset(torch.tensor(test_x), torch.tensor(test_y))
test_loader = DataLoader(dataset=test_set, batch_size=config.batch_size,
                          shuffle=False, num_workers=0)

model = Intere_Gene_Model(config).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr = config.lr)

In [ ]:
print(config.model_type)

Recall_list = defaultdict(list)
HitRate_list = defaultdict(list)

for epoch in range(config.epochs):
    model.train()
    total_loss = 0
    
    '''*********************  training  **************************'''
    for train_loader in tqdm(train_loader_list):
        for data in train_loader:
            data = data[0].cuda()
            optimizer.zero_grad()
            loss = model(data)
            total_loss += loss*1e-6
            loss.backward()
            optimizer.step()
    print("epoch: %d   loss: %10.4f"%(epoch+1,total_loss))
    
    '''*********************  testing  **************************'''
    model.eval()
    test_num = 0
    Recall = defaultdict(float)
    HitRate = defaultdict(float)
    for x, y in tqdm(test_loader):
        x = x.cuda()
        y = y.numpy()
        recall_topN = model.serving(x)
        
        test_num += x.shape[0]
        for N in config.N_list:
            this_N = recall_topN[:,:N]
            for i in range(y.shape[0]):
                Recall[N] += (np.intersect1d(this_N[i],y[i]).shape[0])/y[i].shape[0]
                HitRate[N] += np.intersect1d(this_N[i],y[i]).shape[0] > 0
    
    for N in config.N_list:
        Recall[N] /= test_num
        HitRate[N] /= test_num
        print("Recall%d :  %10.4f    HitRate%d :  %10.4f"%(N,Recall[N],N,HitRate[N]))
        Recall_list[N].append(copy.deepcopy(Recall[N]))
        HitRate_list[N].append(copy.deepcopy(HitRate[N]))

Intere_Gene v1


100%|██████████| 20/20 [01:07<00:00,  3.37s/it]


epoch: 1   loss:   -21.3167


100%|██████████| 61/61 [00:15<00:00,  4.06it/s]


Recall20 :      0.0017    HitRate20 :      0.0073
Recall50 :      0.0026    HitRate50 :      0.0119


...

100%|██████████| 61/61 [00:16<00:00,  3.77it/s]


Recall20 :      0.0142    HitRate20 :      0.0514
Recall50 :      0.0237    HitRate50 :      0.0843


100%|██████████| 20/20 [01:09<00:00,  3.47s/it]


epoch: 23   loss: -1316180608.0000


 69%|██████▉   | 42/61 [00:11<00:05,  3.62it/s]

In [ ]:
torch.save(model, model_save_path + model_name + ".pth") 
torch.save(model.state_dict(),model_save_path + model_name + "_para.pth")

In [ ]:
with open (ans_save_path + model_name +".csv", 'w', encoding='utf8') as f:
    writer  = csv.writer(f)
    for N in config.N_list:
        writer.writerow(Recall_list[N])
        writer.writerow(HitRate_list[N])

In [1]:
import random

In [2]:
l = [2,6,3,5,3,9,0,3]

In [4]:
random.sample(l,3)

[6, 2, 5]